In [ ]:
import yaml
import time
import random
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
import time
from envs.photo_env import PhotoEnhancementEnv
from sac.sac_algorithm import SAC
import multiprocessing as mp
try:
    mp.set_start_method('spawn', force=True)
except RuntimeError:
    pass  


In [ ]:
env = PhotoEnhancementEnv(64)

In [ ]:
with open("configs/hyperparameters.yaml") as f:
    config_dict =yaml.load(f, Loader=yaml.FullLoader)
    
class Config(object):
    def __init__(self, dictionary):
        self.__dict__.update(dictionary)
sac_config = Config(config_dict)

SEED = sac_config.seed
DEVICE= 'CUDA'

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = sac_config.torch_deterministic

In [ ]:
run_name = f"{sac_config.exp_name}__{sac_config.seed}__{int(time.time())}"
writer = SummaryWriter(f"runs/{run_name}")
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(sac_config).items()])),
)
agent = SAC(env,sac_config,writer)

In [ ]:
for i in range(sac_config.total_timesteps):
    episode_count = 0
    agent.start_time = time.time()
    agent.reset_env()
    envs_mean_rewards =[]
    while True:
        
        episode_count+=1
        agent.global_step+=1
        mean_reward,batch_dones = agent.train()
        envs_mean_rewards.append(mean_reward)

        if (batch_dones==True).all()==True or episode_count==10:
            ens_mean_episodic_return = sum(envs_mean_rewards)
            agent.writer.add_scalar("charts/episodic_return", ens_mean_episodic_return, agent.global_step)
            episode_count=0
            envs_mean_rewards =[]
            agent.reset_env()
            break     